### GEOGRAPHIC DATA SCIENCE - Part1 - Introduction to geographic data

Guest lecture by Sándor Juhász | [sandorjuhasz.com](sandorjuhasz.com)
<br>


**Where to drink beer in Budapest?**

Better bars in Buda?  Which district is worth going out in?<br>

In the following we will explore standard geographic data files and learn how to plot, combine and aggregate geographic data.

<img src="../figures/part1_figure.png">

In [ ]:
# if you run on Google Colab
!git clone https://github.com/sandorjuhasz/geoDS_guest_lectures.git
%cd geoDS_guest_lectures/code

In [ ]:
# import libraries
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

*/data/budapest_bars.geojson* contains information on BARS around Budapest (from Google Maps) -- let's use this as an example to start exploring geographic data

In [ ]:
# google data on bars
bp_bars = gpd.read_file("../data/budapest_bars.geojson")
bp_bars

In [ ]:
# plot the data
bp_bars.plot()

Seems like the 'geometry' column contains POINT objects with coordinates and we can very easily visualize a bunch of dots in an abstract space. But it definitely misses context! */data/shape_files/budapest_shape* contains a SHP shape file with the boarders of Budapest.

In [ ]:
# Budapest shape
bp_shape = gpd.read_file("../data/shape_files/budapest_shape/budapest_shape.shp")
bp_shape

In [ ]:
# check out how this 'POLYGON' looks like
bp_shape.plot()

In [ ]:
# this is not a LAKE -- so lets keep its boundary ONLY -- note that the x-y axis run on different levels than the previous plot
bp_shape.boundary.plot()

In [ ]:
# check CRS
bp_bars.crs

In [ ]:
# change the crs!
bp_shape = bp_shape.to_crs(4326)
bp_shape.crs

In [ ]:
# lets combine the two
fig, ax = plt.subplots(1,1, figsize=(8,5))
bp_bars.plot(ax=ax)
bp_shape.boundary.plot(ax=ax)

It looks like these are indeed *bars* around Budapest. Budapest is famously devided by the river Duna. To add more context, we could probably add the waterfronts.

In [ ]:
# rivers and waterfronts from OpenStreetMap -- bunch of different POLYGONs
bp_river_shape = gpd.read_file("../data/shape_files/budapest_waters_shape/budapest_waters_shape.shp")
bp_river_shape

In [ ]:
# lets see how they look like
bp_river_shape.plot()

In [ ]:
# why do we have multiple rows with the name "Duna"
bp_river_shape[(bp_river_shape["osm_id"]=="6712324") | (bp_river_shape["osm_id"]=="187129")].plot(column="osm_id")

It is cool, but usually we work with boring administrative data. So let's bring districts to the map.

In [ ]:
# districts
bp_districts = gpd.read_file("../data/shape_files/budapest_districts_shape/budapest_districts_shape.shp")
bp_districts

Let's combine all this somehow to have a very nice plot on *bars around Budapest*! 

In [ ]:
# Budapest shape
bp_shape = gpd.read_file("../data/shape_files/budapest_shape/budapest_shape.shp")

# river shape
bp_river_shape = gpd.read_file("../data/shape_files/budapest_waters_shape/budapest_waters_shape.shp")

# districts
bp_districts = gpd.read_file("../data/shape_files/budapest_districts_shape/budapest_districts_shape.shp")

# google data on bars
bp_bars = gpd.read_file("../data/budapest_bars.geojson")

fig, ax = plt.subplots(1,1, figsize=(8,6))
bp_river_shape.plot(ax=ax)
bp_districts.boundary.plot(linewidth=0.5, color="grey", ax=ax)
bp_bars.plot(color="#f28e1c", markersize=5, ax=ax)
ax.set_title("Bars around Budapest", size=18)
ax.set_axis_off()

**Best bars around Buda?**

Use *sjoin* (spatial join) to combine the tables and identify bars (POINT) around districts of Buda (POLYGON) from spatial data.

In [ ]:
# look at the tables
bp_bars.head(2)

In [ ]:
bp_districts.head(2)

In [ ]:
# identify bars around Buda through spatial join (sjoin)
bp_bars.sjoin(bp_districts[bp_districts["buda01"]==1])

We must plot it, to be sure that it works!

In [ ]:
# Buda and Pest bars
fig, ax = plt.subplots(1,1, figsize=(8,6))
bp_river_shape.plot(ax=ax)
bp_districts.boundary.plot(linewidth=0.5, color="grey", ax=ax)

# identify bars around Buda through spatial join (sjoin)
buda_bars = bp_bars.sjoin(bp_districts[bp_districts["buda01"]==1])["place_id"].to_list()
bp_bars["buda_bar"] = np.where(bp_bars["place_id"].isin(buda_bars), 1, 0)
bp_bars.plot(column="buda_bar", markersize=5, ax=ax)

ax.set_title("Bars around Buda and Pest", size=18)
ax.set_axis_off()

Looks good! *sjoin* seems to be a very powerful tool! So far we joined POINT and POLYGON type of data. Does it work with a set of POLYGONs?

Let's see if there are bars close to the river Duna in district IX. and XI.

In [ ]:
# data prep
focal_districts = ["IX. kerulet", "XI. kerulet"]
fd = bp_districts[bp_districts["name"].isin(focal_districts)]
fd.plot()

In [ ]:
# join river POLYGON and district POLYGON
river_in_fd = gpd.sjoin(bp_river_shape, fd, how="inner", predicate="intersects")
river_in_fd

In [ ]:
# bars around the river Duna
fd_bars = bp_bars.sjoin(fd)

fig, ax = plt.subplots(1,1, figsize=(8,6))
river_in_fd.plot(ax=ax, zorder=2)
fd.boundary.plot(color="darkgrey", ax=ax, zorder=3)
fd_bars.plot(color="#f28e1c", markersize=5, ax=ax, zorder=4)

ax.set_title("Bars around Duna (IX. and XI.)", size=18)
ax.set_axis_off()

Let's find the best bars around Buda!

In [ ]:
# identify bars around Buda through spatial join (sjoin)
buda_bars = bp_bars.sjoin(bp_districts[bp_districts["buda01"]==1])["place_id"].to_list()
bp_bars["buda_bar"] = np.where(bp_bars["place_id"].isin(buda_bars), 1, 0)

In [ ]:
# define the bests
rating_threshold = 4.9
minimum_rating = 20
buda01 = 1
best_bars = bp_bars[
    (bp_bars["rating"] >= rating_threshold)
    & (bp_bars["nr_ratings"] >= minimum_rating)
    & (bp_bars["buda_bar"] == buda01)
]

bp_bars["best_on_side"] = (bp_bars["place_id"].isin(best_bars["place_id"])).astype(int)

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(6,5))
bp_river_shape.plot(ax=ax)
bp_districts.boundary.plot(linewidth=0.5, color="grey", ax=ax)
bp_bars[bp_bars["best_on_side"]==1].plot(color="red", ax=ax)
ax.set_title("The best bars Buda", size=18)
ax.set_axis_off()

It is good to know that there are cool bars around Buda, but the map of bar dots suggest that Pest has a higher density of bars. Let's make a standard plot to show the number of bars per district.

In [ ]:
# use spatial join to figure out bar and district relations
bars_in_districts = bp_districts.sjoin(bp_bars)

# pandas magic to aggregate data by districts
bars_in_districts = bars_in_districts.groupby(["name", "geometry"]).agg(nr_bars = pd.NamedAgg("place_id", "nunique")).reset_index()

# turn the pandas DataFrame into a GeoDataFrame
bars_in_districts = gpd.GeoDataFrame(bars_in_districts)

# make it stellar!
fig,ax = plt.subplots(1,1, figsize=(6,4))
bars_in_districts.plot(column="nr_bars", cmap="Reds", legend=True, legend_kwds={"label": "Number of bars", "orientation": "vertical"}, ax=ax)
ax.set_axis_off()

In [ ]:
# combine all the above to make things look just NICE
fig, ax = plt.subplots(1,3, figsize=(15, 5))
fontsize=16

bp_river_shape.plot(ax=ax[0])
bp_districts.boundary.plot(linewidth=0.5, color="grey", ax=ax[0])
bp_bars.plot(color="#f28e1c", markersize=5, ax=ax[0])
ax[0].set_title("Bars around Budapest", size=fontsize)
ax[0].set_axis_off()

bp_river_shape.plot(ax=ax[1])
bp_districts.boundary.plot(linewidth=0.5, color="grey", ax=ax[1])
bp_bars[bp_bars["best_on_side"]==1].plot(color="red", ax=ax[1])
ax[1].set_title("The best bars of Buda", size=fontsize)
ax[1].set_axis_off()

bars_in_districts.plot(column="nr_bars", cmap="Reds", ax=ax[2])
bars_in_districts.boundary.plot(color="white", linewidth=0.125, ax=ax[2])
ax[2].axis('off')

# create colorbar as a legend
vmin = bars_in_districts["nr_bars"].min()
vmax = bars_in_districts["nr_bars"].max()

sm = plt.cm.ScalarMappable(cmap="Reds", norm=plt.Normalize(vmin=vmin, vmax=vmax))
cbar = fig.colorbar(sm, orientation="vertical", fraction=0.0325, pad=0.025, shrink=0.9, ax=ax[2])
cbar.ax.set_ylabel("Number of bars", size=fontsize)
cbar.ax.get_yaxis().labelpad = 10
cbar.ax.tick_params(labelsize = fontsize-5)
ax[2].set_axis_off()

#plt.savefig("../figures/part1_figure.png", bbox_inches='tight', facecolor="white")